In [16]:
import random
import pandas as pd
from glob import glob
import numpy as np  
import os

In [43]:
def train_validate_test_split(img_dir, class_label=0, train_percent=.4, validate_percent=.1, base_train_percent=.4, seed=87264):
    img_list = glob(img_dir + '*.dcm')
    img_list = [os.path.join(s.split('/')[-2], s.split('/')[-1]) for s in img_list] #this is really ugly!
    df = pd.DataFrame({'image':img_list, 'label':[class_label]*len(img_list), 'split':['-']*len(img_list)})
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df)
    train_end = int(train_percent * m)
    base_train_end = int(base_train_percent * m) + train_end
    validate_end = int(validate_percent * m) + base_train_end
    df.loc[perm[:train_end], 'split'] = 'train'
    df.loc[perm[train_end:base_train_end], 'split'] = 'base_train'
    df.loc[perm[base_train_end:validate_end], 'split'] = 'val'
    df.loc[perm[validate_end:], 'split'] = 'test'
    return df

In [44]:
df_hr = train_validate_test_split('/project/catinous/cat_data/hr/', class_label=0)
df_hr_cat = train_validate_test_split('/project/catinous/cat_data/hr_cat/', class_label=1)
df_hr = df_hr.append(df_hr_cat)

df_hr.to_csv('catsinom_hr_dataset.csv')

In [45]:
df_lr = train_validate_test_split('/project/catinous/cat_data/lr/', class_label=0)
df_lr_cat = train_validate_test_split('/project/catinous/cat_data/lr_cat/', class_label=1)
df_lr = df_lr.append(df_lr_cat)

df_lr.to_csv('catsinom_lr_dataset.csv')

In [46]:
df = df_lr.append(df_hr)
df.to_csv('catsinom_combined_dataset.csv')

In [47]:
df_lr.sample(10000, replace=True)

,image,label,split
721,lr_cat/26416C06_20130903_dicom_86438.dcm,1,test
1157,lr/23CDE306_20150723_dicom_70216.dcm,0,base_train
1179,lr_cat/CFC6D2FE_20080505_dicom_49442.dcm,1,base_train
278,lr_cat/AE9D4C2D_20111017_dicom_57598.dcm,1,base_train
664,lr_cat/1MY48MCG_20150106_dicom_279577.dcm,1,test
...,...,...,...
826,lr/4d61d27a_20140202_dicom_2793264.dcm,0,test
894,lr/8A09FF04_20091119_dicom_76889.dcm,0,base_train
1485,lr_cat/97675744_20130518_dicom_44500.dcm,1,train
1516,lr_cat/D1381C6B_20130521_dicom_61588.dcm,1,train
